# O criptosistema de Boneh e Fanklin 

Foi o primeiro esquema IBE a ser aceito como um IBE adequado ao uso industrial

Em 2007 foi standartizado no [RFC 5091](https://tools.ietf.org/html/rfc5091)  e em 2015 no ISO/IEC  18033-5:2015.

A versão aqui apresentada tem objetivo meramente pedagógico e não deve ser usada fora deste contexto. Uma sugestão de implementação mais robusta é a biblioteca [pbc](https://crypto.stanford.edu/pbc/)

## Parte A - Criar a Estrutura Algébrica necessária ao **BF**

### 1º Passo
Gerar uma curva supersingular aleatória sobre um corpo primo $\mathbb{Z}_p$.

+ As curvas super-singulares sobre $\mathbb{Z}_p$  são as curvas da forma $\;Y^2 = X^3 + b$.

+ Para a curva ser super-singular, a característica  $p$ tem de verificar $p = 2 \bmod 3$

In [1]:
# Gerar a característica "p"

p = next_prime(2^224)
while p %3 != 2:
    p = next_prime(p+2)
b = next_prime(ZZ.random_element(3,p-3))

In [2]:
# Definir o corpo, a curva e verificar a sua cardinalidade

K = GF(p)
E = EllipticCurve(K,[0,b])
print E

# Verificar a cardinalidade
E.cardinality() == p + 1

Elliptic Curve defined by y^2 = x^3 + 22643272660895900053896763422973949047145673798157238498922779202809 over Finite Field of size 26959946667150639794667015087019630673637144422540572481103610250573


True

### 2º Passo
Gerar um ponto $G$ com ordem prima definidor do grupo cíclico  $\lbrack G \rbrack$, a **órbita** de $G$

In [9]:
# a. Gerar um ponto aleatório da curva "P"  cuja ordem "N" tenha um factor primo máximo

def largest_prime_factor(q):
    if is_prime(q):
        return q
    else:
        (n,_) = list(factor(q))[-1]
        return n

n = largest_prime_factor(p+1)    # o maior factor primo da cardinalidade da curva
                                 # este "n" vai ser a ordem do grupo cíclico;
while True:
    P = E.random_element()
    N = P.order()
    if n == largest_prime_factor(N):
        break

In [10]:
# b.  encontrar o gerador "G".
# Se a ordem de "P" for prima encontramos o ponto;  senão determina-se o cofactor "= N//n" e 
# gera-se "G" como P vezes o cofactor

if is_prime(N):
    G = P
else:
    cof = N//n
    G = P*cof

# Teste
print G*n == E(0)

True


In [11]:
# c. Ver a ordem do grupo cíclico gerado por "G"
    
print "Tamanho da ordem de G = %d bits" % floor(RDF(log(n,2)))

Tamanho da ordem de G = 164 bits


In [12]:
G.weil_pairing(G,n)

1

### 3º Passo

Determinar o grau de embebimento para a ordem $n$; isto é, 

o menor valor $k$ tal que o corpo $\text{GF}(p^k)$ contém todas as $n$-raizes da unidade

Uma propriedade das curvas singulares é que esse grau de embebimento só pode ser 1 ou 2

In [5]:
if gcd(p-1,n) == 1:
    k = 2
else:
    k = 1
    
print k

2


### 4º Passo
+ Definir a estensão do corpo base e um gerador nesse corpo com a mesma ordem $n$
+ Definir a curva estendida e verificar a sua cardinalidade

In [13]:
# Corpo estendido
Kx = GF(p^k,name='z')

# Curva estendida e o ponto "G" na nova curva

Ex = EllipticCurve(Kx,[0,b])
Gx = Ex(G.xy())

# Verificar a cardinalidade
Ex.cardinality() == (p+1)^2

True

In [10]:
Gx.weil_pairing(Gx,n)

1

### 5º Passo

Para definir um emarelhamento não degenerado é necessário definir um mapeamento  de pontos na curva que seja um isomorfismo e preserve a estrutura de grupo abeliano; tais mapeamentos designam-se por **isogenias**.
 
A tranformação tem de mapear pontos de `Ex` em pontos de `Ex` com a mesma ordem.

A transformação escolhida é   $(x,y) \mapsto (z3*x , y)$ em que $z3$ é uma raiz cúbica da unidade em `Fx` que seja diferente de $1$; isto é:  uma raiz do polinómio $X^2+X+1$ em `Kx`.

In [14]:
# Calcular "z3"
Fx.<X> = Kx[]
(z3,_) = Fx(X^2+X+1).roots()[0]



# Definir o mapeamento
def phi(P):
    if P == Ex(0):
        return P
    (x,y) = P.xy()
    return Ex(z3*x , y)

# Verificar se têm a mesm ordem e se é uma isogenia
print phi(Gx).order() == n
i = ZZ.random_element(1,n-1)
print phi(Gx*i) == phi(Gx)*i

True
True


### 6º Passo  
### definição do emparelhamento

Para definir o emparelhamento vamos tomar por base o emparelhamento de Weil que o Sage tem programado na função `weil_pairing`. 

O emparelhamento de Weil, implementado `weil_pairing`,  mapeia um par de pontos em `Ex` numa $n$-raíz da unidade em `Fx`.

No entanto este emparelhamento aplicado a um par de múltiplos de um mesmo ponto $G$ produz sempre $1$ (é degenerado).

Para evitar este problema aplica-se a um dos argumentos a transformação `phi` determinada antes.

In [15]:
def pairing(P,Q):
    return P.weil_pairing(phi(Q),n)
    
# Verificar que é um emparelhamento não degenerado, no grupo cíclico gerado por Gx

g = pairing(Gx,Gx) 
print g != 1
print pairing(Gx*3 , Gx*4) == g^12

print g.multiplicative_order() == n


True
True
True


## Parte B - Implementação do BF

### 1º Passo 
+ Definição e implementação das funções de 'hash'.
+ Definição e implementação de funções auxiliares para manipulação da informação

Para programar o criptosistema de Boneh e Franklin são necessárias 4 funções de `hash`.
As três primeira têm como domínio strings de bytes arbitrárias
+ `ID` -- mapeia no grupo cíclico gerado por `Gx` (excepto `Ex(0)`)
+ `H `  -- é um hash standard; vamos usar o `sha256` da biblioteca **hashlib** mapeados nos inteiros do `Sage` (o anel `ZZ`).
+ `Hn` -- mapeia num inteiro no intervalo `Zn`  
+ `fconv` é um hash de conversão de grupo cíclico gerado por `g` para o output de `H` (256 bits)

A forma mais conveniente de lidar com estas funções (e outras manipulações usadas no BF) é através de uma classe `Data`; como métodos desta classe temos os vários hash's, e as operações de `xor` e  concatenação em `arrays` de `bytes`.

In [10]:
import sys, os, hashlib, types, binascii
import numpy as np

class Data(object):
    def __init__(self,arg):
        
        if isinstance(arg,np.ndarray) and arg.dtype == np.dtype('uint8'):
            self.array = arg
            self.data  = self.array.tobytes()
        else:
            if isinstance(arg, types.StringTypes):
                self.data  = arg
            elif isinstance(arg, Integer):
                self.data  = os.urandom(arg)
            else:
                self.data =  str(arg)
            self.array = np.array(self.data,"c").view(np.uint8)           
        self.len = len(self.data)
        self._hash = hashlib.sha256(self.data)
        
    def __str__(self):
        return binascii.hexlify(self.data)
        
    def xor(self,other):
        if not isinstance(other,Data):
            raise TypeError("argument of type %s is not Data" % type(other))
        if self.len < other.len:
            return Data(np.bitwise_xor(self.array,other.array[:self.len]))
        elif other.len < self.len:
            return Data(np.bitwise_xor(self.array[:other.len],other.array))
        else:
            return Data(np.bitwise_xor(self.array,other.array))
        
    def pair(self,other):
        if not isinstance(other,Data):
            raise TypeError("argument of type %s is not Data" % type(other))
        return Data(self.data + other.data)
        
    def pref(self,i):
        return Data(self.data[:i])
        
    def iH(self):
        return ZZ(self._hash.hexdigest(),16)
        
    def H(self):
        return Data(self._hash.digest())
        
    def ID(self):
        return Gx * (self.iH())
     
    def Hn(self):
        return (self.iH()) % n


In [11]:
# A função "fconv" aparece de forma diferente: não como um método da classe Data 
# mas usando os objetos dessa classe      
def fconv(s):
    return Data(s).H()

# testes e exemplos

# exemplo de uma chave pública em IBE

w = Data("Jose Silva#email jose.silva@gmail.com#code X1JK0B#start 2017-01-01#end 2017-12-31")

# Vários hash's de "w"

print w.iH()
print w.H()
print w.ID()

print fconv(Gx)

# Manipulação de dados

u = Data(16)
v = Data(16)
print u.xor(v)
print u.pair(v)

110766301506293960422297788738826869251089097096418771804904440529019996876480
f4e381445c451aeef3049c4ad66e5e2960e20e7776ea7200022c987640ca2ec0
(6606536503625789246060915408709516180453836939320371064481717441456 : 1552917644213642366109271489820905181204610851286952069435045889619 : 1)
9c3da213fac85cc94267876faab7cc3e23bf11898097d016b3de8a70050687bc
889b2d241100cdb26fbc2f96a4530e96
f8f0754e689799706fcf7bb730f54322706b586a799754c20073542194a64db4


### 2º Passo

Implementação do BF como um KEM (_key encapsulation mechanism_)

In [12]:
class BF(object):
    def __init__(self):
        self.k = ZZ.random_element(1,n-1)    # chaves de autenticação
        self.beta = Gx * (self.k)
     
    def extract(self,w):
        id = w.ID()
        return id * (self.k)
        
    def validate(self,w,privKey):
        id = w.ID() 
        if pairing(self.beta,id) != pairing(Gx,privKey):
            raise ValueError("A chave privada não é adequada à chave pública")
        
    def KEM(self,w):
        id = w.ID()
        r = Data(32) ; key = r.H() 
        tag = Data(16); a = (tag.pair(r)).Hn() ; 
        alpha = Gx * a ; u = pairing(self.beta,id)^a ; f = fconv(u)
        crypt = (alpha, r.xor(f),tag.xor(key))
        return (key.data, crypt)
        
    def deKEM(self,crypt,privKey):
        (alpha, r1 , tag1) = crypt
        u = pairing(alpha,privKey) ; f = fconv(u) ; r = r1.xor(f) 
        key = r.H() ; tag = tag1.xor(key); a = (tag.pair(r)).Hn()
        if alpha == Gx * a:
            return key.data
        else:
            raise ValueError("A chave revelada não é autêntica")

### 3º Passo - Teste

In [13]:
# Verificar a chave pública
print "|%s|" % w.data
print

# Criar uma instância do criptosistema
ibe = BF()

# extrair a chave privada
pK = ibe.extract(w)

# validar a chave
try:
    ibe.validate(w,pK)

# Cifrar    
    (key, cr) = ibe.KEM(w)
# Decifrar
    key1 = ibe.deKEM(cr,pK)

# Verificação
    if  key != key1:
        raise ValueError("A chave revelada é incorrecta")
    print "OK"
    
except Exception as err:
    print err

|Jose Silva#email jose.silva@gmail.com#code X1JK0B#start 2017-01-01#end 2017-12-31|

OK
